In [1]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from tqdm.notebook import tqdm

from trl import SFTTrainer

/home/u17a883cf852b9f81bb6e3bb6ee3b080/.conda/envs/itrex/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/u17a883cf852b9f81bb6e3bb6ee3b080/.conda/envs/itrex/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


2024-06-29 00:30:23.094391: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-29 00:30:24.075599: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-29 00:30:27.477218: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from huggingface_hub import interpreter_login

In [3]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: fineGrained).
Your token has been saved to /home/u17a883cf852b9f81bb6e3bb6ee3b080/.cache/huggingface/token
Login successful


In [4]:
dataset = load_dataset("llamafactory/tiny-supervised-dataset", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 300
})

In [6]:
import pandas as pd
# Convert to DataFrame
df = pd.DataFrame(dataset)

# Display the first few rows of the DataFrame
df.head(2)

,instruction,input,output
0,Give three tips for staying healthy.,,1. Eat a balanced diet and make sure to includ...
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye..."


In [7]:
# Function to transform the row into desired format
def format_row(row):
    question = row['instruction']
    answer = row['output']
    formatted_string = f"[INST] {question} [/INST] {answer} "
    return formatted_string

# Apply the function to each row of the dataframe
df['Formatted'] = df.apply(format_row, axis=1)

# Display the formatted column
df['Formatted']

0      [INST] Give three tips for staying healthy. [/...
1      [INST] What are the three primary colors? [/IN...
2      [INST] Describe the structure of an atom. [/IN...
3      [INST] How can we reduce air pollution? [/INST...
4      [INST] Pretend you are a project manager of a ...
                             ...                        
295    [INST] 生成创新性解决方案以减少食品浪费。 [/INST] 减少食品浪费的一种可能的解...
296    [INST] 总结下面提供的信息。 [/INST] River Technologies在2...
297    [INST] 创建一个数据库来存储下面给出的用户数据。 [/INST] 可以创建以下数据库来...
298    [INST] 讨论给定COVID-19疫苗对国际贸易的潜在影响。 [/INST] COVID...
299    [INST] 提供三种方法来改善工作场所的沟通。 [/INST] 1. 鼓励开放性对话：鼓励...
Name: Formatted, Length: 300, dtype: object

In [8]:
# Rename the 'Formatted' column to 'Text'
new_df = df.rename(columns={'Formatted': 'Text'})

new_df

,instruction,input,output,Text
0,Give three tips for staying healthy.,,1. Eat a balanced diet and make sure to includ...,[INST] Give three tips for staying healthy. [/...
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye...",[INST] What are the three primary colors? [/IN...
2,Describe the structure of an atom.,,"An atom is made up of a nucleus, which contain...",[INST] Describe the structure of an atom. [/IN...
3,How can we reduce air pollution?,,There are a number of ways to reduce air pollu...,[INST] How can we reduce air pollution? [/INST...
4,Pretend you are a project manager of a constru...,,I had to make a difficult decision when I was ...,[INST] Pretend you are a project manager of a ...
...,...,...,...,...
295,生成创新性解决方案以减少食品浪费。,,减少食品浪费的一种可能的解决方案是创建一个在线平台，作为食品企业和家庭之间的中介。在这样的平...,[INST] 生成创新性解决方案以减少食品浪费。 [/INST] 减少食品浪费的一种可能的解...
296,总结下面提供的信息。,与2017年第四季度相比，River Technologies的销售额在2018年第一季度下...,River Technologies在2018年第一季度报告销售额下降了6.2％，与2017...,[INST] 总结下面提供的信息。 [/INST] River Technologies在2...
297,创建一个数据库来存储下面给出的用户数据。,用户名\n密码\n邮箱\n电话号码,可以创建以下数据库来存储用户数据：\n表名：users\n字段：username（字符串）、...,[INST] 创建一个数据库来存储下面给出的用户数据。 [/INST] 可以创建以下数据库来...
298,讨论给定COVID-19疫苗对国际贸易的潜在影响。,,COVID-19疫苗的开发有可能对国际贸易产生积极影响。随着各国努力为其公民接种疫苗，对基本...,[INST] 讨论给定COVID-19疫苗对国际贸易的潜在影响。 [/INST] COVID...


In [9]:
new_df = new_df[['Text']]
new_df.head(3)

,Text
0,[INST] Give three tips for staying healthy. [/...
1,[INST] What are the three primary colors? [/IN...
2,[INST] Describe the structure of an atom. [/IN...


In [10]:
new_df.to_csv('formatted_data.csv', index=False)

In [11]:
final_df = pd.read_csv("formatted_data.csv")

In [12]:
final_df.head(2)

,Text
0,[INST] Give three tips for staying healthy. [/...
1,[INST] What are the three primary colors? [/IN...


In [13]:
training_dataset = load_dataset("csv", data_files="formatted_data.csv", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
training_dataset

Dataset({
    features: ['Text'],
    num_rows: 300
})

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
import torch

base_model = "NousResearch/Llama-2-7b-chat-hf"
new_model = "Llama-2-mental-health"

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

training_arguments = TrainingArguments(
    output_dir='./tmp',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    overwrite_output_dir=True,
    per_device_train_batch_size=8,  # Increased batch size
    per_device_eval_batch_size=8,   # Increased batch size
    gradient_accumulation_steps=1,  # Reduced gradient accumulation steps
    save_strategy="no",
    log_level="info",
    save_total_limit=2,
    bf16=True,
    no_cuda=True  # Disable CUDA (GPU) usage
)

peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
)

trainer = SFTTrainer(
    model=model,
    train_dataset=training_dataset,  # Training datadet being used here
    peft_config=peft_config,
    dataset_text_field="Text",
    max_seq_length=500,
    tokenizer=tokenizer,
    args=training_arguments,
)


/home/u17a883cf852b9f81bb6e3bb6ee3b080/.conda/envs/itrex/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/u17a883cf852b9f81bb6e3bb6ee3b080/.conda/envs/itrex/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [16]:
trainer.train()

/home/u17a883cf852b9f81bb6e3bb6ee3b080/.conda/envs/itrex/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 300
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 114
  Number of trainable parameters = 33,554,432
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=114, training_loss=1.486392104834841, metrics={'train_runtime': 1313.9695, 'train_samples_per_second': 0.685, 'train_steps_per_second': 0.087, 'total_flos': 1.1887313760288768e+16, 'train_loss': 1.486392104834841, 'epoch': 3.0})

In [17]:
from transformers import pipeline  

In [20]:
import os
os.environ['USE_IPEX'] = '0'

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

base_model = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model)

prompt = "How can we reduce air pollution?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])


loading file tokenizer.model from cache at /home/u17a883cf852b9f81bb6e3bb6ee3b080/.cache/huggingface/hub/models--NousResearch--Llama-2-7b-chat-hf/snapshots/351844e75ed0bcbbe3f10671b3c808d2b83894ee/tokenizer.model
loading file tokenizer.json from cache at /home/u17a883cf852b9f81bb6e3bb6ee3b080/.cache/huggingface/hub/models--NousResearch--Llama-2-7b-chat-hf/snapshots/351844e75ed0bcbbe3f10671b3c808d2b83894ee/tokenizer.json
loading file added_tokens.json from cache at /home/u17a883cf852b9f81bb6e3bb6ee3b080/.cache/huggingface/hub/models--NousResearch--Llama-2-7b-chat-hf/snapshots/351844e75ed0bcbbe3f10671b3c808d2b83894ee/added_tokens.json
loading file special_tokens_map.json from cache at /home/u17a883cf852b9f81bb6e3bb6ee3b080/.cache/huggingface/hub/models--NousResearch--Llama-2-7b-chat-hf/snapshots/351844e75ed0bcbbe3f10671b3c808d2b83894ee/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/u17a883cf852b9f81bb6e3bb6ee3b080/.cache/huggingface/hub/models--NousResearc

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at NousResearch/Llama-2-7b-chat-hf.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /home/u17a883cf852b9f81bb6e3bb6ee3b080/.cache/huggingface/hub/models--NousResearch--Llama-2-7b-chat-hf/snapshots/351844e75ed0bcbbe3f10671b3c808d2b83894ee/generation_config.json
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "pad_token_id": 32000,
  "temperature": 0.9,
  "top_p": 0.6,
  "transformers_version": "4.31.0"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.31.0"
}



[INST] How can we reduce air pollution? [/INST]  Reducing air pollution requires a multi-faceted approach that involves individual actions, government policies, and technological innovations. Here are some ways to reduce air pollution:

1. Use public transportation, walk, or bike: Using public transportation, walking, or biking instead of driving can significantly reduce air pollution caused by vehicle emissions.
2. Use energy-efficient appliances: Using energy-efficient appliances can reduce energy consumption, which can lead to lower emissions.
3. Reduce, reuse, recycle: Reduce consumption, reuse products, and recycle materials to reduce waste and lower emissions.
4. Plant trees and greenery: Trees and other greenery help absorb pollutants from the air, reducing air pollution.
5. Use clean energy sources: Transitioning to clean energy sources like solar, wind, and hydroelectric power can reduce emissions from fossil fuels.
6. Implement emissions controls: Governments can implement em

In [1]:
import os
os.environ['USE_IPEX'] = '0'

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

base_model = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model)

prompt = "Generate a poem that expresses anger."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

2024-06-29 07:45:36.233663: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-29 07:45:37.230579: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-29 07:45:40.591267: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/u17a883cf852b9f81bb6e3bb6ee3b080/.conda/envs/itrex/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be remove

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/u17a883cf852b9f81bb6e3bb6ee3b080/.conda/envs/itrex/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[INST] Generate a poem that expresses anger. [/INST]  Sure, here's a poem that expresses anger:

Raging fire in my veins,
A tempest raging out of bounds,
Fury in my heart, a stormy sea,
Unleashed, untamed, and wild as can be.

I feel the heat, the pressure builds,
My soul is consumed by bitter pills,
The injustice, the betrayal, the pain,
A never-ending cycle, a never-ending strain.

I'm trapped in this cage of rage,
A prisoner of my own disgrace,
The world outside, a blurred haze,
As I struggle to find my way to peace.

But peace is hard to find,
When anger's all that's on my mind,
A never-ending battle, a war within,
A fight to keep my sanity in check, to begin.

So I'll let the anger flow,
Like a river that won't let go,
For in its depths,
